In [18]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [19]:
es = Elasticsearch(['https://192.168.1.181:9200'],
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))
searchContext = Search(using=es, index='*:so-*', doc_type='doc')

In [20]:
s = searchContext.query('query_string', query='destination.port:443')
#s=searchContext.query('query_string', query='message')
#s= searchContext.query('query_string', query='message:("ts" AND "id.orig_h" AND "id.resp_h" AND "host" AND "id.resp_p" AND "method")')
#s = searchContext.query('query_string', query='event.category:conn')

In [21]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)
response = s.execute()


#base response query
response = s.execute()
if response.success():
  df = pd.DataFrame((d.to_dict() for d in s.scan()))
df.head(20)

,metadata,data,log,destination,rule,syslog,source,interface,ingest,network,ecs,host,@version,event,tcp,ip,message,tags,@timestamp,firewall,destination_geo,server,observer,client,connection,ssl,hash,notice,intel,file,weird,agent,process,winlog,user,source_geo
0,"{'pipeline': 'syslog', 'raw_index': 'so-syslog', 'beat': 'filebeat', 'truncated': False, 'ip_address': '172.17.0.1', 'type': '_doc', 'version': '7.13.4'}",{'length': '0'},{'source': {'address': '192.168.1.154:514'}},"{'geo': {'continent_name': 'North America', 'region_iso_code': 'US-WA', 'city_name': 'Seattle', 'country_iso_code': 'US', 'timezone': 'America/Los_Angeles', 'ip': '13.249.109.26', 'country_name': 'United States', 'region_name': 'Washington', 'location': {'lon': -122.3451, 'lat': 47.6348}}, 'port': '443', 'ip': '13.249.109.26'}","{'reason': 'match', 'sub_uuid': '', 'action': 'block', 'uuid': '8'}","{'priority': '134', 'timestamp': 'Nov 23 18:50:10'}","{'application': 'filterlog', 'port': '40228', 'ip': '192.168.120.89'}",{'name': 'vmx2'},{'timestamp': '2021-11-23T23:50:10.753Z'},"{'community_id': '1:IqEPTI7jkNXEVL00s9dM0FnfRSg=', 'transport_id': '6', 'transport': 'tcp', 'direction': 'in'}",{'version': '1.8.0'},{'name': 'sohunter'},1,"{'module': 'pfsense', 'dataset': 'firewall'}",{'flags': 'S'},"{'ecn': '', 'offset': '0', 'flags': 'DF', 'tos': '0x0', 'id': '60005', 'version': '4', 'ttl': '64'}","<134>Nov 23 18:50:10 filterlog[16437]: 8,,,1000000103,vmx2,match,block,in,4,0x0,,64,60005,0,DF,6,tcp,60,192.168.120.89,13.249.109.26,40228,443,0,S,291846052,,64240,,mss;sackOK;TS;nop;wscale",[beats_input_codec_plain_applied],2021-11-23T18:50:10.000Z,"{'tracker_id': '1000000103', 'anchor': ''}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'pipeline': 'syslog', 'raw_index': 'so-syslog', 'beat': 'filebeat', 'truncated': False, 'ip_address': '172.17.0.1', 'type': '_doc', 'version': '7.15.2'}",{'length': '0'},{'source': {'address': '192.168.1.154:514'}},"{'geo': {'continent_name': 'North America', 'country_iso_code': 'US', 'timezone': 'America/Chicago', 'ip': '13.32.214.71', 'country_name': 'United States', 'location': {'lon': -97.822, 'lat': 37.751}}, 'port': '443', 'ip': '13.32.214.71'}","{'reason': 'match', 'sub_uuid': '', 'action': 'block', 'uuid': '8'}","{'priority': '134', 'timestamp': 'Nov 27 23:46:59'}","{'application': 'filterlog', 'port': '55292', 'ip': '192.168.120.89'}",{'name': 'vmx2'},{'timestamp': '2021-11-28T04:46:59.194Z'},"{'community_id': '1:eC5VeXoTTXO2ukmM2P6h5yIR6DU=', 'transport_id': '6', 'transport': 'tcp', 'direction': 'in'}",{'version': '1.11.0'},{'name': 'sohunter'},1,"{'module': 'pfsense', 'dataset': 'firewall'}",{'flags': 'S'},"{'ecn': '', 'offset': '0', 'flags': 'DF', 'tos': '0x0', 'id': '53163', 'version': '4', 'ttl': '64'}","<134>Nov 27 23:46:59 filterlog[16437]: 8,,,1000000103,vmx2,match,block,in,4,0x0,,64,53163,0,DF,6,tcp,60,192.168.120.89,13.32.214.71,55292,443,0,S,515481303,,64240,,mss;sackOK;TS;nop;wscale",[beats_input_codec_plain_applied],2021-11-27T23:46:59.000Z,"{'tracker_id': '1000000103', 'anchor': ''}","{'ip': '13.32.214.71', 'organization_name': 'AMAZON-02', 'asn': 16509, 'network': '13.32.208.0/20'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'pipeline': 'syslog', 'raw_index': 'so-syslog', 'beat': 'filebeat', 'truncated': False, 'ip_address': '172.17.0.1', 'type': '_doc', 'version': '7.15.2'}",{'length': '0'},{'source': {'address': '192.168.1.154:514'}},"{'geo': {'continent_name': 'North America', 'country_iso_code': 'US', 'timezone': 'America/Chicago', 'ip': '18.64.236.31', 'country_name': 'United States', 'location': {'lon': -97.822, 'lat': 37.751}}, 'port': '443', 'ip': '18.64.236.31'}","{'reason': 'match', 'sub_uuid': '', 'action': 'block', 'uuid': '8'}","{'priority': '134', 'timestamp': 'Dec 1 19:15:23'}","{'application': 'filterlog', 'port': '54992', 'ip': '192.168.120.89'}",{'name': 'vmx2'},{'timestamp': '2021-12-02T00:15:23.270Z'},"{'community_id': '1:xw5LppRDaxM27EID9Cm84S4JE6

In [22]:
json_struct = json.loads(df.to_json(orient="records"))
df_flat = json_normalize(json_struct)
df_flat.head(5)

,@version,message,tags,@timestamp,destination_geo,server,observer,client,connection,ssl,hash,notice,intel,file,weird,agent,process,winlog,user,source_geo,metadata.pipeline,metadata.raw_index,metadata.beat,metadata.truncated,metadata.ip_address,metadata.type,metadata.version,data.length,log.source.address,destination.geo.continent_name,destination.geo.region_iso_code,destination.geo.city_name,destination.geo.country_iso_code,destination.geo.timezone,destination.geo.ip,destination.geo.country_name,destination.geo.region_name,destination.geo.location.lon,destination.geo.location.lat,destination.port,destination.ip,rule.reason,rule.sub_uuid,rule.action,rule.uuid,syslog.priority,syslog.timestamp,source.application,source.port,source.ip,interface.name,ingest.timestamp,network.community_id,network.transport_id,network.transport,network.direction,ecs.version,host.name,event.module,event.dataset,tcp.flags,ip.ecn,ip.offset,ip.flags,ip.tos,ip.id,ip.version,ip.ttl,firewall.tracker_id,firewall.anchor,destination_geo.ip,destination_geo.organization_name,destination_geo.asn,destination_geo.network,data,rule,syslog,interface,host,tcp,ip,firewall,log.file.path,log.offset,log.id.uid,event.category,server.port,server.ip,server.ip_bytes,server.packets,observer.name,client.port,client.ip,client.ip_bytes,client.packets,connection.state_description,connection.bytes.missed,connection.state,connection.history,connection.local.responder,connection.local.originator,network,ssl.cipher,ssl.established,ssl.server_name,ssl.curve,ssl.certificate.chain_fuids,ssl.certificate.subject,ssl.certificate.issuer,ssl.client.certificate.chain_fuids,ssl.validation_status,ssl.resumed,ssl.version,hash.ja3s,hash.ja3,ssl.next_protocol,network.bytes,event.duration,server.bytes,client.bytes,network.protocol,log.id.fuid,notice.p,notice.suppress_for,notice.note,notice.sub_message,notice.peer_description,notice.action,notice.message,ssl.last_alert,intel.indicator,intel.sources,intel.seen_where,intel.indicator_type,intel.matched,intel.seen_node,file.description,file.mimetype,weird.peer,weird.name,weird.notice,ingest,log.level,destination.hostname,source.hostname,event.code,event.provider,event.kind,event.created,event.action,agent.hostname,agent.name,agent.id,agent.ephemeral_id,agent.type,agent.version,process.pid,process.entity_id,process.executable,winlog.computer_name,winlog.process.pid,winlog.process.thread.id,winlog.channel,winlog.event_data.SourcePortName,winlog.event_data.DestinationPortName,winlog.event_data.UtcTime,winlog.event_data.Initiated,winlog.event_data.SourceIsIpv6,winlog.event_data.DestinationIsIpv6,winlog.event_data.RuleName,winlog.opcode,winlog.version,winlog.record_id,winlog.event_id,winlog.task,winlog.provider_guid,winlog.api,winlog.provider_name,winlog.user.identifier,winlog.user.domain,winlog.user.name,winlog.user.type,user.name,rule.severity,rule.reference,rule.rev,rule.metadata.performance_impact,rule.metadata.affected_product,rule.metadata.attack_target,rule.metadata.updated_at,rule.metadata.former_category,rule.metadata.created_at,rule.metadata.signature_severity,rule.metadata.deployment,rule.gid,rule.name,rule.ruleset,rule.rule,rule.category,network.data.decoded,event.severity,event.severity_label,source.geo.continent_name,source.geo.region_iso_code,source.geo.city_name,source.geo.country_iso_code,source.geo.timezone,source.geo.ip,source.geo.country_name,source.geo.region_name,source.geo.location.lon,source.geo.location.lat,source_geo.ip,source_geo.organization_name,source_geo.asn,source_geo.network
0,1,"<134>Nov 23 18:50:10 filterlog[16437]: 8,,,1000000103,vmx2,match,block,in,4,0x0,,64,60005,0,DF,6,tcp,60,192.168.120.89,13.249.109.26,40228,443,0,S,291846052,,64240,,mss;sackOK;TS;nop;wscale",[beats_input_codec_plain_applied],2021-11-23T18:50:10.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,syslog,so-syslog,filebeat,False,172.17.0.1,_doc,7.13.4,0,192.168.1.154:514,North America,US-WA,Seattle,US,America/Los_Angeles,13.249.109.26,U

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406458 entries, 0 to 406457
Data columns (total 36 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   metadata         406458 non-null  object
 1   data             123117 non-null  object
 2   log              406458 non-null  object
 3   destination      406458 non-null  object
 4   rule             123118 non-null  object
 5   syslog           123117 non-null  object
 6   source           406458 non-null  object
 7   interface        123117 non-null  object
 8   ingest           406213 non-null  object
 9   network          310436 non-null  object
 10  ecs              406458 non-null  object
 11  host             123363 non-null  object
 12  @version         406458 non-null  object
 13  event            406458 non-null  object
 14  tcp              120542 non-null  object
 15  ip               123117 non-null  object
 16  message          406458 non-null  object
 17  tags      